# Monte Carlo Simulation

In [1]:
from classes.MyDataLogger import MyDataLogger, MySubDataLogger
from modules.helper_functions_tsp import (find_problem_size,
                                          cost_fn_fact, 
                                          find_distances_array,
                                          convert_integer_to_binary_list,
                                          )
from itertools import count
import random


In [2]:
from modules.config import (GRAY,
                            DECODING_FORMULATION
                            )

Instantiate datalogger

In [3]:
datalogger = MyDataLogger()
sdl_list = []

Define constants:

In [4]:
LOCATIONS = [15,17,26,42,48,15,17,26,42,48,15,17,26,42,48,15,17,26,42,48,]
ITERATIONS = 1
QUANTUM = True

Validate input

In [5]:
if not isinstance(QUANTUM, bool):
    raise TypeError("QUANTUM must be a Boolean (True or False)")

In [6]:
#entered manually from show_results.ipynb and results table
#maximum cache size for Circuits 1, 2, 3, 4, 6 by location
if QUANTUM:
   cache_size = {4 : 8_016,
               5 : 30_336,
               6 : 170_262,
               7 : 407_199,
               8 : 807_210,
               9 : 953_795,
               10 : 1_014_396,
               11 : 1_012_825,
               12 : 1_025_235,
               15 : 1_026_048,
               17 : 1_026_048,
               26 : 1_026_048,
               42 : 1_026_048, 
               48 : 1_026_048,
                  }
else:
   cache_size = {4 : 1_028_096,
               5 : 1_542_144,
               6 : 2_313_216,
               7 : 3_084_288,
               8 : 3_855_360,
               9 : 4_626_432,
               10 : 5_654_528,
               11 : 6_682_624,
               12 : 7_710_720,
               15 : 10_795_008,
               17 : 12_851_200,
               26 : 24_417_280,
               42 : 47_292_416, 
               48 : 56_545_280,
                  }

Main loop

In [7]:
print(f'Running Monte Carlo simulations for {DECODING_FORMULATION} formulation with Gray = {GRAY}.')
print(f'Running with Quantum = {QUANTUM}.')
for iterations in range(ITERATIONS):
    print(f'Iteration {iterations} of {ITERATIONS}.')
    for i in range(len(LOCATIONS)):
        sdl = MySubDataLogger(runid = datalogger.runid)
        sdl.quantum = QUANTUM
        sdl.locations = LOCATIONS[i]
        sdl.formulation = DECODING_FORMULATION
        sdl.gray = GRAY
        sdl.hot_start = False
        sdl.monte_carlo = True
        qubits = find_problem_size(sdl)
        sdl.qubits = qubits
        print(f'There are {qubits} qubits needed for {sdl.locations} locations in the {sdl.formulation} formulation.')
        distance_array, best_dist = find_distances_array(sdl.locations, 
                                                        print_comments=False,
                                                        )
        sdl.best_dist = best_dist 
        cost_fn = cost_fn_fact(sdl,
                                distance_array, 
                            )
        sdl.cache_max_size = cache_size[sdl.locations]
        print(f'Setting cache max size to {sdl.cache_max_size} for {sdl.locations} locations.')
        lowest_to_date = 999_999 #too high
        highest = 2**sdl.qubits-1
        for j in range(sdl.cache_max_size):
            integer = random.randint(1, highest)
            binary_string_list = convert_integer_to_binary_list(integer, 
                                                                sdl.qubits, 
                                                                gray = sdl.gray)
            cost = cost_fn(binary_string_list)
            if cost < lowest_to_date:
                lowest_to_date = cost
                iterations_found = j
        
        print(f' The lowest cost for {sdl.locations} locations is {lowest_to_date} which was found after {iterations_found} iterations out of {sdl.cache_max_size}.')
        sdl.best_dist_found = lowest_to_date
        sdl.update_cache_statistics(cost_fn)
        print(f'Cache hits: {sdl.cache_hits}, cache misses: {sdl.cache_misses}, total cache calls: {sdl.cache_hits + sdl.cache_misses}.')
        sdl.save_results_to_csv()

Running Monte Carlo simulations for original formulation with Gray = False.
Running with Quantum = True.
Iteration 0 of 1.
SubDataLogger instantiated.  Run ID = 20260201-11-42-58 - 11-42-59
There are 41 qubits needed for 15 locations in the original formulation.
Reading distance data
Setting cache max size to 1026048 for 15 locations.
 The lowest cost for 15 locations is 385.0 which was found after 622025 iterations out of 1026048.
Cache hits: 0, cache misses: 1026048, total cache calls: 1026048.
Saving data to results\results.csv
SubDataLogger instantiated.  Run ID = 20260201-11-42-58 - 11-44-09
There are 49 qubits needed for 17 locations in the original formulation.
Reading distance data
Setting cache max size to 1026048 for 17 locations.
 The lowest cost for 17 locations is 2579.0 which was found after 246326 iterations out of 1026048.
Cache hits: 0, cache misses: 1026048, total cache calls: 1026048.
Saving data to results\results.csv
SubDataLogger instantiated.  Run ID = 20260201-1